<a href="https://colab.research.google.com/github/DhrubaAdhikary/ERA_V2/blob/master/S21_Training_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00


In [6]:
# !python /content/train_get2-9-speedup9.py


In [8]:

import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANGPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y


class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 256  # reduced max sequence length to fit into 4GB GPU
    vocab_size: int = 50304  # number of tokens
    n_layer: int = 6  # increased number of layers for better learning
    n_head: int = 8  # increased number of heads for better learning
    n_embd: int = 256  # increased embedding dimension for better learning


class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        param_dict = {pn: p for pn, p in self.named_parameters() if p.requires_grad}
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"using device: {device}")

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

num_return_sequences = 5
max_length = 30

import tiktoken

class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T
        with open('input.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'loaded {len(self.tokens)} tokens')
        print(f'1 epoch = {len(self.tokens) // (B * T)} batches')
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position: self.current_position + B * T + 1]
        x = (buf[:-1]).view(B, T)
        y = (buf[1:]).view(B, T)
        self.current_position += B*T
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

torch.set_float32_matmul_precision('high')
model = GPT(GPTConfig())
model.to(device)

max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 50  # increased warmup steps
max_steps = 1000  # increased max steps

def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

train_loader = DataLoaderLite(B=4, T=256)

import time

optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device)

num_epochs = 1
accumulation_steps = 4

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for step in range(max_steps):
        t0 = time.time()
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits, loss = model(x, y)
        loss.backward()
        norm = torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            lr = get_lr(step)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        t1 = time.time()
        dt = t1 - t0
        print(f"Step {step + 1}/{max_steps}, Loss: {loss.item():.4f}, Time: {dt * 1000:.2f} ms")

using device: cuda
loaded 338025 tokens
1 epoch = 330 batches
Epoch 1/1
Step 1/1000, Loss: 10.8596, Time: 18.97 ms


<ipython-input-8-c413576343d3>:209: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  norm = torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)


Step 2/1000, Loss: 10.8898, Time: 15.01 ms
Step 3/1000, Loss: 10.8724, Time: 14.18 ms
Step 4/1000, Loss: 10.8408, Time: 17.93 ms
Step 5/1000, Loss: 10.3939, Time: 14.90 ms
Step 6/1000, Loss: 10.2975, Time: 14.00 ms
Step 7/1000, Loss: 10.3101, Time: 14.42 ms
Step 8/1000, Loss: 10.2598, Time: 14.82 ms
Step 9/1000, Loss: 10.2861, Time: 14.10 ms
Step 10/1000, Loss: 10.2660, Time: 14.82 ms
Step 11/1000, Loss: 10.2019, Time: 14.77 ms
Step 12/1000, Loss: 10.3912, Time: 14.28 ms
Step 13/1000, Loss: 10.1505, Time: 13.87 ms
Step 14/1000, Loss: 10.1383, Time: 15.51 ms
Step 15/1000, Loss: 10.1753, Time: 13.80 ms
Step 16/1000, Loss: 10.2071, Time: 17.75 ms
Step 17/1000, Loss: 10.1107, Time: 14.15 ms
Step 18/1000, Loss: 10.0649, Time: 14.63 ms
Step 19/1000, Loss: 10.0208, Time: 14.64 ms
Step 20/1000, Loss: 10.0421, Time: 17.49 ms
Step 21/1000, Loss: 9.8443, Time: 14.30 ms
Step 22/1000, Loss: 10.0723, Time: 14.19 ms
Step 23/1000, Loss: 9.8215, Time: 14.55 ms
Step 24/1000, Loss: 9.8278, Time: 16.72 ms

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import time
import tiktoken  # Assuming this is a utility for tokenization

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y


class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 256  # reduced max sequence length to fit into 4GB GPU
    vocab_size: int = 50304  # number of tokens
    n_layer: int = 6  # increased number of layers for better learning
    n_head: int = 8  # increased number of heads for better learning
    n_embd: int = 256  # increased embedding dimension for better learning


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            wpe=nn.Embedding(config.block_size, config.n_embd),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        param_dict = {pn: p for pn, p in self.named_parameters() if p.requires_grad}
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

# Initialize model and prepare for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)
model = GPT(GPTConfig()).to(device)
optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device)
train_loader = DataLoaderLite(B=4, T=256)  # Adjust batch size and sequence length as needed

# Training loop
num_epochs = 20
max_steps = 1000

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for step in range(max_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits, loss = model(x, y)
        loss.backward()
        optimizer.step()
        print(f"Step {step + 1}/{max_steps} | Loss: {loss.item():.4f}")

print("Training completed successfully.")


Streaming output truncated to the last 5000 lines.
Step 6/1000 | Loss: 1.8202
Step 7/1000 | Loss: 1.5345
Step 8/1000 | Loss: 1.9093
Step 9/1000 | Loss: 1.6938
Step 10/1000 | Loss: 1.9894
Step 11/1000 | Loss: 1.7140
Step 12/1000 | Loss: 1.8872
Step 13/1000 | Loss: 1.7128
Step 14/1000 | Loss: 2.1265
Step 15/1000 | Loss: 2.2721
Step 16/1000 | Loss: 2.1492
Step 17/1000 | Loss: 2.0144
Step 18/1000 | Loss: 1.7331
Step 19/1000 | Loss: 2.0080
Step 20/1000 | Loss: 1.8211
Step 21/1000 | Loss: 1.7549
Step 22/1000 | Loss: 1.7236
Step 23/1000 | Loss: 1.7665
Step 24/1000 | Loss: 1.8104
Step 25/1000 | Loss: 1.9676
Step 26/1000 | Loss: 2.2003
Step 27/1000 | Loss: 2.1039
Step 28/1000 | Loss: 2.0842
Step 29/1000 | Loss: 2.1093
Step 30/1000 | Loss: 2.0263
Step 31/1000 | Loss: 2.0068
Step 32/1000 | Loss: 1.8059
Step 33/1000 | Loss: 2.0369
Step 34/1000 | Loss: 2.0976
Step 35/1000 | Loss: 1.9823
Step 36/1000 | Loss: 1.9558
Step 37/1000 | Loss: 1.9057
Step 38/1000 | Loss: 1.6564
Step 39/1000 | Loss: 1.4387
S

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import time
import tiktoken  # Assuming this is a utility for tokenization
from dataclasses import dataclass

# Set float32 matmul precision
torch.set_float32_matmul_precision('high')

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANGPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y


class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 1024  # reduced max sequence length to fit into 4GB GPU
    vocab_size: int = 50304  # number of tokens
    n_layer: int = 6  # increased number of layers for better learning
    n_head: int = 8  # increased number of heads for better learning
    n_embd: int = 256  # increased embedding dimension for better learning


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            wpe=nn.Embedding(config.block_size, config.n_embd),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        param_dict = {pn: p for pn, p in self.named_parameters() if p.requires_grad}
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

# Initialize model and prepare for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)
model = GPT(GPTConfig()).to(device)
optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device)
train_loader = DataLoaderLite(B=4, T=256)  # Adjust batch size and sequence length as needed

# Cosine learning rate decay schedule
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 1000

def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for step in range(max_steps):
        t0 = time.time()
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        # Automatic mixed precision
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits, loss = model(x, y)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Adjust learning rate
        lr = get_lr(step)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        optimizer.step()
        torch.cuda.synchronize() if device == 'cuda' else None
        t1 = time.time()
        dt = (t1 - t0) * 1000
        tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)

        print(f'Step {step + 1}/{max_steps} | Loss: {loss.item():.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}')

print("Training completed successfully.")


loaded 338025 tokens
1 epoch = 330 batches
Epoch 1/20
Step 1/1000 | Loss: 10.8735 | dt: 113.56ms | tok/sec: 9017.13
Step 2/1000 | Loss: 10.5486 | dt: 87.81ms | tok/sec: 11661.85
Step 3/1000 | Loss: 10.3371 | dt: 62.83ms | tok/sec: 16298.14
Step 4/1000 | Loss: 10.0952 | dt: 62.22ms | tok/sec: 16457.52
Step 5/1000 | Loss: 10.1463 | dt: 61.43ms | tok/sec: 16668.87
Step 6/1000 | Loss: 9.9253 | dt: 67.80ms | tok/sec: 15104.03
Step 7/1000 | Loss: 9.8955 | dt: 70.71ms | tok/sec: 14482.48
Step 8/1000 | Loss: 9.7390 | dt: 61.32ms | tok/sec: 16698.16
Step 9/1000 | Loss: 9.6423 | dt: 58.76ms | tok/sec: 17427.13
Step 10/1000 | Loss: 9.4429 | dt: 64.62ms | tok/sec: 15846.54
Step 11/1000 | Loss: 9.1802 | dt: 57.99ms | tok/sec: 17657.69
Step 12/1000 | Loss: 9.2253 | dt: 59.65ms | tok/sec: 17166.62
Step 13/1000 | Loss: 8.8134 | dt: 63.11ms | tok/sec: 16224.87
Step 14/1000 | Loss: 8.7492 | dt: 59.37ms | tok/sec: 17249.21
Step 15/1000 | Loss: 8.5228 | dt: 61.83ms | tok/sec: 16562.61
Step 16/1000 | Loss: